# Restoring a 3D point from two observations through triangulation

In this recipe, you will learn how to reconstruct 3D point coordinates given observations in two views. This is a building block for many higher level 3D reconstruction algorithms and SLAM systems.

We generate random points in the 3D space and project them into two test views. Then, we add noise to those observations and reconstruct points back in 3D using the OpenCV function `cv2.triangulatePoints`. As input, the function takes observations from two cameras and camera projection matrices (projective mapping from the world coordinate frame to a view coordinate frame) for each view. It returns the reconstructed points in the world coordinate frame.

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
import os

%matplotlib auto
%pylab inline

def print_image(header,name,np_arr,
                start_First=0,end_First=1,start_Second=0,end_Second=2,start_3=0,end_3=5):
    print("------  {0:-<25}    Shape{1} {2}: {3:}".format(header, np_arr.shape, name, str(np_arr.dtype)) )
    shapes = np_arr.shape #print(shapes)
    if shapes[0] < end_First:
        end_First = shapes[0]
    if shapes[1] < end_Second:
        end_Second = shapes[1]
    if len(shapes)==3:
        if shapes[2] < end_3:
            end_3 = shapes[2]
    if len(shapes)==3:
        for i in range (start_First,end_First):
            print("[", sep='',end="")
            for j in range (start_Second,end_Second):
                print(np_arr[i,j,start_3:end_3], sep=' ', end=" ")
            print(']')
    if len(shapes)==2:
        for i in range (start_First,end_First):
            print("[", end=" ")
            #print(np_arr[i,start_Second:end_Second],sep=' ',end=" ") cutoff sting by<60
            for k in range (start_Second,end_Second):
                print(np_arr[i,k], end=" ")
            print(']')

def draw_grid(img, pxystep=None,major_color=None, pxstep=None,pystep=None):
    #print("{0} XY{1} color{2} X{3} Y{4}".format(img.shape, pxystep,major_color,pxstep,pystep))
    pXYstep = None; pXstep=None; pYstep=None; 
    major_Color=None; minor_Color=None; major_Alpha=None; minor_Alpha=None;
    if pxystep != None:
        pXYstep = pXstep = pYstep = pxystep;
    else:
        pXstep = pxstep if pxstep != None else 100
        pYstep = pystep if pystep != None else 100
    major_Color = major_color if major_color != None else (204, 204, 204) #'#CCCCCC'
    if pXstep != None:
        x = pXstep
        #Draw all lines on X
        while x < img.shape[1]:
            cv2.line(img, (x, 0), (x, img.shape[0]), color=major_Color, thickness=1)
            x += pXstep
    if pYstep != None:
        y = pYstep
        #Draw all lines on Y
        while y < img.shape[0]:
            cv2.line(img, (0, y), (img.shape[1], y), color=major_Color,thickness=1)
            y += pYstep
    return img

def plt_view_image(plt,list_images,figsize=(15,6), axis="off", cmap='gray'):
    plt.figure(figsize=figsize)
    n = len(list_images)  #; print(n)
    plot_number = 1
    for name, img in list_images:
        plt.subplot(1,n,plot_number)
        plt.axis(axis); plt.title(name)
        if cmap =='gray': plt.imshow(img,cmap='gray' )
        else: plt.imshow(img)
        plot_number = plot_number + 1
    plt.show()

def plt_view_grid(plt, axis ='off',
                  xy_sizeaxis =None,
                  xy_measuare =None,
                  x_min=-10, x_max=10, y_min=-10, y_max=10,
                  x_major_size=1, x_minor_size=0.2, y_major_size=1, y_minor_size=0.2,
                  major_color='#CCCCCC', major_alpha=0.5,
                  minor_color='#CCCCCC', minor_alpha=0.2
                 ):
    if xy_sizeaxis is None:  x_min=-10; x_max=10; y_min=-10; y_max=10;
    else: x_min, x_max, y_min, y_max = xy_sizeaxis

    if xy_measuare is None:  x_major_size=1; x_minor_size=0.2; y_major_size=1; y_minor_size=0.2;
    else: x_major_size, x_minor_size, y_major_size, y_minor_size = xy_measuare
        
    plt.xlim(x_min, x_max); plt.ylim(y_min, y_max);
    ax = plt.gca()
    x_major_ticks=np.arange(x_min,x_max,x_major_size); x_minor_ticks=np.arange(x_min,x_max,x_minor_size)
    y_major_ticks=np.arange(y_min,y_max,y_major_size); y_minor_ticks=np.arange(y_min,y_max,y_minor_size)
    ax.set_xticks(x_major_ticks)
    ax.xaxis.set_major_locator(MultipleLocator(x_major_size))
    ax.set_xticks(x_minor_ticks, minor=True)
    ax.set_yticks(y_major_ticks)
    ax.yaxis.set_major_locator(MultipleLocator(y_major_size))
    ax.set_yticks(y_minor_ticks, minor=True)
    plt.grid(which='major', color=major_color, alpha=major_alpha)
    plt.grid(which='minor', color=minor_color, alpha=minor_alpha)
    #plt.gca().invert_yaxis() plt.gca().invert_xaxis()
    
#help("modules")   
import sys             
print('\n'.join(sys.path))
print("current folder ==",os.getcwd())
#pip list

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib
D:\HTML_DOC\Program\opencv\Packt\S09\env
C:\Program Files\Python38\python38.zip
C:\Program Files\Python38\DLLs
C:\Program Files\Python38\lib
C:\Program Files\Python38
d:\html_doc\program\opencv\packt\s09\env

d:\html_doc\program\opencv\packt\s09\env\lib\site-packages
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\pip-20.1-py3.8.egg
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\win32
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\win32\lib
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\Pythonwin
d:\html_doc\program\opencv\packt\s09\env\lib\site-packages\IPython\extensions
C:\Users\polit\.ipython
current folder == D:\HTML_DOC\Program\opencv\Packt\S09\env


In [2]:
P1 = np.eye(3, 4, dtype=np.float32)
P2 = np.eye(3, 4, dtype=np.float32)
P2[0, 3] = -1


#############################################################
print_image('P1','P1',P1,0,7,0,7)
print_image('P2','P2',P2,0,7,0,7)

------  P1-----------------------    Shape(3, 4) P1: float32
[ 1.0 0.0 0.0 0.0 ]
[ 0.0 1.0 0.0 0.0 ]
[ 0.0 0.0 1.0 0.0 ]
------  P2-----------------------    Shape(3, 4) P2: float32
[ 1.0 0.0 0.0 -1.0 ]
[ 0.0 1.0 0.0 0.0 ]
[ 0.0 0.0 1.0 0.0 ]


In [3]:
np.random.seed(42)

N = 5
points3d = np.empty((4, N), np.float32)
points3d[:3, :] = np.random.randn(3, N)
points3d[3, :] = 1



#############################################################
print_image('points3d','points3d',points3d,0,7,0,7)

------  points3d-----------------    Shape(4, 5) points3d: float32
[ 0.49671414 -0.1382643 0.64768857 1.5230298 -0.23415338 ]
[ -0.23413695 1.5792128 0.7674347 -0.46947438 0.54256004 ]
[ -0.46341768 -0.46572974 0.24196227 -1.9132802 -1.7249179 ]
[ 1.0 1.0 1.0 1.0 1.0 ]


In [4]:
np.random.seed(42)

points1 = P1 @ points3d
points1 = points1[:2, :] / points1[2, :]
points1[:2, :] += np.random.randn(2, N) * 1e-2



#############################################################
print_image('P1','P1',P1,0,7,0,7)
print_image('points3d','points3d',points3d,0,7,0,7)
print_image('points1','points1',points1,0,7,0,7)

------  P1-----------------------    Shape(3, 4) P1: float32
[ 1.0 0.0 0.0 0.0 ]
[ 0.0 1.0 0.0 0.0 ]
[ 0.0 0.0 1.0 0.0 ]
------  points3d-----------------    Shape(4, 5) points3d: float32
[ 0.49671414 -0.1382643 0.64768857 1.5230298 -0.23415338 ]
[ -0.23413695 1.5792128 0.7674347 -0.46947438 0.54256004 ]
[ -0.46341768 -0.46572974 0.24196227 -1.9132802 -1.7249179 ]
[ 1.0 1.0 1.0 1.0 1.0 ]
------  points1------------------    Shape(2, 5) points1: float32
[ -1.0668827 0.29549402 2.6832933 -0.7808004 0.13340601 ]
[ 0.5028982 -3.375043 3.1793869 0.24068195 -0.30911693 ]


In [5]:
points2 = P2 @ points3d
points2 = points2[:2, :] / points2[2, :]
points2[:2, :] += np.random.randn(2, N) * 1e-2



#############################################################
print_image('P2','P2',P2,0,7,0,7)
print_image('points3d','points3d',points3d,0,7,0,7)
print_image('points2','points2',points2,0,7,0,7)

------  P2-----------------------    Shape(3, 4) P2: float32
[ 1.0 0.0 0.0 -1.0 ]
[ 0.0 1.0 0.0 0.0 ]
[ 0.0 0.0 1.0 0.0 ]
------  points3d-----------------    Shape(4, 5) points3d: float32
[ 0.49671414 -0.1382643 0.64768857 1.5230298 -0.23415338 ]
[ -0.23413695 1.5792128 0.7674347 -0.46947438 0.54256004 ]
[ -0.46341768 -0.46572974 0.24196227 -1.9132802 -1.7249179 ]
[ 1.0 1.0 1.0 1.0 1.0 ]
------  points2------------------    Shape(2, 5) points2: float32
[ 1.0813967 2.4393873 -1.4536397 -0.2925009 0.6982361 ]
[ 0.49961674 -3.4009633 3.1748548 0.23629645 -0.32866555 ]


In [6]:
points3d_reconstr = cv2.triangulatePoints(P1, P2, points1, points2)
points3d_reconstr /= points3d_reconstr[3, :]



#############################################################
print_image('P1','P1',P1,0,7,0,7)
print_image('P2','P2',P2,0,7,0,7)
print_image('points1','points1',points1,0,7,0,7)
print_image('points2','points2',points2,0,7,0,7)

------  P1-----------------------    Shape(3, 4) P1: float32
[ 1.0 0.0 0.0 0.0 ]
[ 0.0 1.0 0.0 0.0 ]
[ 0.0 0.0 1.0 0.0 ]
------  P2-----------------------    Shape(3, 4) P2: float32
[ 1.0 0.0 0.0 -1.0 ]
[ 0.0 1.0 0.0 0.0 ]
[ 0.0 0.0 1.0 0.0 ]
------  points1------------------    Shape(2, 5) points1: float32
[ -1.0668827 0.29549402 2.6832933 -0.7808004 0.13340601 ]
[ 0.5028982 -3.375043 3.1793869 0.24068195 -0.30911693 ]
------  points2------------------    Shape(2, 5) points2: float32
[ 1.0813967 2.4393873 -1.4536397 -0.2925009 0.6982361 ]
[ 0.49961674 -3.4009633 3.1748548 0.23629645 -0.32866555 ]


In [7]:
print('Original points')
print(points3d[:3].T)
print('Reconstructed points')
print(points3d_reconstr[:3].T)

Original points
[[ 0.49671414 -0.23413695 -0.46341768]
 [-0.1382643   1.5792128  -0.46572974]
 [ 0.64768857  0.7674347   0.24196227]
 [ 1.5230298  -0.46947438 -1.9132802 ]
 [-0.23415338  0.54256004 -1.7249179 ]]
Reconstructed points
[[ 0.49662217 -0.23332939 -0.4654879 ]
 [-0.13780867  1.580262   -0.46642414]
 [ 0.64861894  0.7679889   0.24172477]
 [ 1.5990037  -0.48839998 -2.0478864 ]
 [-0.23603106  0.5644843  -1.7700291 ]]
